# **此程式為鳥群演算法(PSO)應用實作**
#####   
### **\#我這邊挑選三項進行測試，分別是【DE JONG FUNCTION N. 5】、【RASTRIGIN】及【ROSENBROCK】**

<img src="problems.png" alt="drawing" width="1500"/></img>

### **\#更多訊息可在 https://www.sfu.ca/~ssurjano/optimization.html 找到，裡面還有許多用來測試最佳化的Test Function**
------------------------

## **定義問題**

In [1]:
class DJ5:

    name='DJ5'
    dim=2
    upperbound= 65.536
    lowerbound= -65.536

    a=[[-32., -16.,   0.,  16.,  32., -32., -16.,   0.,  16.,  32., -32.,
        -16.,   0.,  16.,  32., -32., -16.,   0.,  16.,  32., -32., -16.,
          0.,  16.,  32.],
       [-32., -32., -32., -32., -32., -16., -16., -16., -16., -16.,   0.,
          0.,   0.,   0.,   0.,  16.,  16.,  16.,  16.,  16.,  32.,  32.,
         32.,  32.,  32.]]

In [2]:
class Rastrigin:

    name='Rastrigin'
    dim=10
    upperbound= 5.12
    lowerbound= -5.12

In [3]:
class Rosenbrock:

    name='Rosenbrock'
    dim=10
    upperbound= 2.048
    lowerbound= -2.048

-------------------
## **PSO 開始**

In [4]:
import sys
import os
import math
import random
import numpy as np

### **1.定義各問題的fitness如何計算(可參照一開始的圖)**

In [5]:
def fitness(problem,x):
    match problem.name:
        case 'DJ5':
            for i in range(25):
                sumterm2 = (x[0] - problem.a[0][i])**6
                sumterm3 = (x[1]- problem.a[1][i])**6
                total_sum = np.sum(1 / (i + sumterm2 + sumterm3))
            y = 1 / (0.002 + total_sum)
            return y
        #--------------------    
        case 'Rastrigin':
            sum=0
            for i in range(len(x)):
                sum+=x[i]**2-(10*np.cos(2*np.pi*x[i]))
            y = 10*len(x)+sum
            return y
        #--------------------       
        case 'Rosenbrock':
            sum=0
            for i in range(len(x)-1):
                sum+=100*(x[i+1]-x[i]**2)**2 + (x[i]-1)**2  
            y = sum
            return y

### **2.初始化 & 各參數介紹**
###### 
* #### Num_particle = 鳥群數量
* #### particle = 鳥的位置。 例如當問題維度dim=2，particle[0] (第一隻鳥) 可能位於 [1.3 , 0.8] 這個點
* #### p_fitness = 鳥的位置代入 f(x) 計算後得到的數值
* #### p_best_pos = (鳥的) 歷史最佳位置
* #### p_best_fitness = (鳥的) 歷史最佳 f(x)
* #### l_best_pos = 鄰居歷史最佳位置 f(x)
* #### l_best_fitness = 鄰居歷史最佳 f(x)
* #### v = (可以理解為) 鳥的飛行速度
* #### vmax = 速限，太大或太小都可能影響鳥尋找全域最佳解

In [6]:
Num_particle=50

In [7]:
def init(num,dim,ub,lb):
    
    
    particle = [[0 for x in range(dim)] for y in range(Num_particle)]
    p_fitness = [sys.float_info.max]*Num_particle
    
    p_best_pos =[[0 for x in range(dim)] for y in range(Num_particle)]
    p_best_fitness = [sys.float_info.max]*Num_particle
    
    l_best_pos= [[0 for x in range(dim)] for y in range(Num_particle)]
    l_best_fitness = [sys.float_info.max]*Num_particle
    
    v=[[0 for x in range(dim)] for y in range(Num_particle)]
    vmax=(ub - lb) * 0.15
    
    for i in range(Num_particle):
        for j in range(dim):
            rd=random.uniform(0,1)
            #在上下限中隨機給定鳥的速率
            particle[i][j] = lb + rd * (ub - lb)
            v[i][j] = rd * (ub - lb) * 0.1
        
    return particle, p_fitness, p_best_pos, p_best_fitness, l_best_pos, l_best_fitness, v, vmax

### **3.訓練(飛行，尋找解的過程)**
###### 
#### **這邊採用的是Lbest (ring)。**
##### **圖源 https://link.springer.com/article/10.1007/s00521-019-04527-9**
<img src="topology.png" alt="drawing" width="700"/></img>

In [8]:
def fly(problem,maxcycle):

    #初始化
    particle, p_fitness, p_best_pos, p_best_fitness,l_best_pos, l_best_fitness, v, vmax = init(Num_particle,problem.dim,problem.upperbound,problem.lowerbound)
    
    #這裡的gbest是用來回傳結果的，並不是拿來訓練鳥指標
    gbest_fitness=sys.float_info.max
    gbest_pos=[sys.float_info.max]*Num_particle

    #開始飛行
    for currentcycle in range(round(maxcycle/Num_particle)):

        #計算每一隻鳥當前的fitness，並更新歷史最佳位置
        for i in range(Num_particle):
            p_fitness[i] = fitness(problem,particle[i])
            if (p_fitness[i] < p_best_fitness[i]):
                p_best_fitness[i] = p_fitness[i]
                for j in range(problem.dim):
                    p_best_pos[i][j] = particle[i][j]

        #---lbest---
        # 開始向鄰居確認
        for i in range(Num_particle):

            #向左右兩邊各6位鄰居索討訊息
            nei = 6
              
            #起始鄰居是第幾隻鳥
            head = i - nei

            #超出索引界限處理
            if (head < 0):
                head = Num_particle+(i - nei)
                
            #鄰居最佳位置
            best_bird = 0
            #鄰居最佳fitness
            findmin = sys.float_info.max
            
            for j in range(2*nei+1):
                current_bird = head+j
                #超出索引界限處理
                if head+j >= Num_particle:
                    current_bird=Num_particle-(head+j)
                #找鄰居中最好的fitness(包含自己)
                if (p_best_fitness[current_bird] < findmin):
                    findmin = p_best_fitness[current_bird]
                    #更新最好的鄰居位置
                    best_bird = current_bird
                    
            #紀錄歷史最佳鄰居位置
            for j in range(problem.dim):
                l_best_pos[i][j] = p_best_pos[best_bird][j]
            #紀錄歷史最佳鄰居fitness   
            l_best_fitness[i]=findmin
            
            #若鄰居最佳勝過全域，更新全域fitness及位置
            if (l_best_fitness[i] < gbest_fitness):
                gbest_fitness = l_best_fitness[i]
                gbest_pos=l_best_pos[i]
                
            #更新鳥的飛行速度與位置
            rd=random.uniform(0,1) 
            for j in range(problem.dim):
                v[i][j] = v[i][j] * 0.7298 + 1.496 * rd* (p_best_pos[i][j] - particle[i][j]) + 1.496 * rd * (l_best_pos[i][j] - particle[i][j])
                if (v[i][j] > vmax):
                    v[i][j] = vmax
                if (v[i][j] < -vmax):
                    v[i][j] = -vmax
                particle[i][j] = particle[i][j] + v[i][j]
                if (particle[i][j] > problem.upperbound ):
                    particle[i][j] = problem.upperbound
                if (particle[i][j] < problem.lowerbound):
                    particle[i][j] = problem.lowerbound    
    return gbest_fitness,gbest_pos

### **4.求解**

In [9]:
problems=[DJ5(),Rastrigin(),Rosenbrock()]

for i in problems:
    #總共要run幾回合
    num_Repetitive_Runs=3

    #全域最佳fitness
    AllGlobalBestFitness = sys.float_info.max
    
    #擁有全域最佳fitness的 鳥的位置
    AllGlobalsolution=[]

    for run in range(num_Repetitive_Runs):
        #飛行
        run_Best_fitness, solution=fly(i,60000)

        #當這回合的fitness勝過全域最佳時，更新全域最佳
        if (run_Best_fitness < AllGlobalBestFitness ):
            AllGlobalBestFitness = run_Best_fitness
            AllGlobalsolution = solution
            
    print("問題:",i.name,", 全域最佳解f(x)=",round(AllGlobalBestFitness,1),", 此時x=:",[round(x,2) for x in AllGlobalsolution])
    print("-----------------------------------")

問題: DJ5 , 全域最佳解f(x)= 22.9 , 此時x=: [32.0, 32.0]
-----------------------------------
問題: Rastrigin , 全域最佳解f(x)= 2.3 , 此時x=: [-0.02, -0.0, 0.0, -0.98, 0.0, 0.02, 0.01, 1.01, -0.01, 0.01]
-----------------------------------
問題: Rosenbrock , 全域最佳解f(x)= 0.0 , 此時x=: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.99]
-----------------------------------


-------------------------
## **\#討論**
###### 
#### **根據開頭的網址，可以找到除了 DE JONG FUNCTION N. 5以外的兩function的最佳解，分別為:**
* #### **Rastrigin**
><img src="Rastrigin_minimum.jpg" alt="drawing" width="200"/></img>
* #### **Rosenbrock**
><img src="Rosenbrock_minimum.jpg" alt="drawing" width="200"/></img>

#### **對比結果會發現仍有一點距離，可能的改善方式大概有:**
#### **1.調整鳥群數量、迭代次數**
#### **2.採用Gbest+Lbest的做法，亦或是其他變種優化過的PSO，本文就不再贅述，感謝觀看**